In [53]:
import requests
import json
from requests.auth import HTTPBasicAuth

# Registration and creation of team

### 1. Follow instructions to create [AAP test accout](https://submission-test.ebi.ac.uk/api/docs/guide_accounts_and_logging_in.html) or [AAP production account](https://submission.ebi.ac.uk/api/docs/guide_accounts_and_logging_in.html)

In [54]:
# Assign your AAP username and accout to these variables
AAP_USERNAME = ""
AAP_PASSWORD = ""

In [104]:
# Comment out these lines if you don't want to use test server for submission
aap_server = 'https://explore.api.aai.ebi.ac.uk/auth'
submission_server = 'https://submission-test.ebi.ac.uk/api/'

# Uncomment these line if you want to use production server
# aap_server = 'https://api.aai.ebi.ac.uk/auth'
# submission_server = 'https://submission.ebi.ac.uk/api'

### 2. Use this function to get token (valid for 1 hour only)

In [105]:
def get_token():
    response = requests.get(aap_server, auth=HTTPBasicAuth(AAP_USERNAME, AAP_PASSWORD))
    return response.text

In [106]:
TOKEN = get_token()

### 3. Create a team

In [107]:
# Provide team description in this line
team_description = ""

# Provide centre name description in this line
centre_name = ""
team_data = {
    'description': team_description,
    'centreName': centre_name
}
team_data = json.dumps(team_data)

In [33]:
create_team_response = requests.post(f'{submission_server}user/teams',
                                     headers={'Content-Type': 'application/json',
                                              'Accept': 'application/hal+json',
                                              'Authorization': f'Bearer {TOKEN}'},
                                     data=team_data)

In [35]:
# You need to get new token after team creation
TOKEN = get_token()

# Create submission

### 1. Start from the root

In [57]:
root_response = requests.get(submission_server, headers={'Content-Type': 'application/json',
                                                         'Accept': 'application/hal+json',
                                                         'Authorization': f'Bearer {TOKEN}'})

### 2. Peak a team

In [59]:
team_response = requests.get(root_response.json()['_links']['userTeams']['href'],
                             headers={'Content-Type': 'application/json',
                                      'Accept': 'application/hal+json',
                                      'Authorization': f'Bearer {TOKEN}'})

### 3. Creating a submission

In [61]:
# We choose first team here (index 0)
create_submissions_link = team_response.json()['_embedded']['teams'][0]['_links']['submissions:create']['href']

In [62]:
creating_submission_response = requests.post(create_submissions_link, headers={'Content-Type': 'application/json',
                                                                               'Accept': 'application/hal+json',
                                                                               'Authorization': f'Bearer {TOKEN}'}, 
                                             data='{}')

### 4. Adding a sample record to a submission

In [64]:
submission_contents_link = creating_submission_response.json()['_links']['contents']['href']

In [65]:
submission_contents_response = requests.get(submission_contents_link, headers={'Content-Type': 'application/json',
                                      'Accept': 'application/hal+json',
                                      'Authorization': f'Bearer {TOKEN}'})

In [66]:
samples_create_link = submission_contents_response.json()['_links']['samples:create']['href']

In [67]:
# Please provide path to samples.json file that you got from validation system, ex. /Users/alexey/samples.json
samples_json_path = ''
with open(samples_json_path, 'r') as f:
    data = json.loads(f.read())

In [68]:
for record in data:
    record = json.dumps(record)
    create_sample_response = requests.post(samples_create_link, headers={'Content-Type': 'application/json',
                                                                         'Accept': 'application/hal+json',
                                                                         'Authorization': f'Bearer {TOKEN}'}, 
                                           data=record)

### 5. Finalising your submission

In [75]:
submission_link = root_response.json()['_links']['userSubmissions']['href']

In [76]:
submission_response = requests.get(submission_link, headers={'Content-Type': 'application/json',
                                                             'Accept': 'application/hal+json',
                                                             'Authorization': f'Bearer {TOKEN}'})

In [77]:
submission_status_link = submission_response.json()['_embedded']['submissions'][0]['_links']['submissionStatus']['href']

In [78]:
submission_status_response = requests.get(submission_status_link, headers={'Content-Type': 'application/json',
                                                                           'Accept': 'application/hal+json',
                                                                           'Authorization': f'Bearer {TOKEN}'})

In [79]:
self_link = submission_status_response.json()['_links']['self']['href']

In [80]:
self_response = requests.get(self_link, headers={'Content-Type': 'application/json',
                                                 'Accept': 'application/hal+json',
                                                 'Authorization': f'Bearer {TOKEN}'})

In [81]:
available_statuses_link = self_response.json()['_links']['availableStatuses']['href']

In [82]:
available_statuses_response = requests.get(available_statuses_link, headers={'Content-Type': 'application/json',
                                                                             'Accept': 'application/hal+json',
                                                                             'Authorization': f'Bearer {TOKEN}'})

In [85]:
change_submission_status_response = requests.put(submission_status_link, headers={'Content-Type': 'application/json',
                                                                                  'Accept': 'application/hal+json',
                                                                                  'Authorization': f'Bearer {TOKEN}'},
                                                 data='{"status": "Submitted"}')

### 6. Get status of submission

In [86]:
processing_statuses_link = submission_response.json()['_embedded']['submissions'][0]['_links']['processingStatuses']['href']
processing_statuses_link = processing_statuses_link + '&size=500'
processing_statuses_response = requests.get(processing_statuses_link, headers={'Content-Type': 'application/json',
                                                                               'Accept': 'application/hal+json',
                                                                               'Authorization': f'Bearer {TOKEN}'})
with open('submission_results.txt', 'w') as f:
    for sample in processing_statuses_response.json()['_embedded']['processingStatuses']:
        if 'accession' in sample:
            f.write(f"{sample['alias']}\t{sample['accession']}\t{sample['status']}\n")
        else:
            print(f"sample accession is not available yet for {sample['alias']}, repeat this cell in several seconds")